## This notebook does variable selections based on codebook review
Note: Daily surveys haven't been selected yet

In [1]:
import sys, os
DMOL_DIR = "/Users/Henryye/research/shaf/DMOL"
sys.path.append(os.path.join(DMOL_DIR, "utils"))

import pandas as pd
import numpy as np
import re
from ml_utils import save_object, load_object

from collections import defaultdict

In [2]:
df = pd.read_excel("../data/student_interested_var_decision_modified.xlsx", sheet_name = None)

### Pre & Post

In [3]:
pre_post_vars = defaultdict(lambda: defaultdict(set))
for var, con in zip(df["Pre & Post"]["Var"], df["Pre & Post"]["Construct"]):
    if var is not np.nan:
        if "pre" in var:
            pre_post_vars["pre"][con].add(var)
        elif "post" in var:
            pre_post_vars["post"][con].add(var)
        else:
            pre_post_vars["other"][con].add(var)

In [4]:
# Course Identifiers
pre_post_vars["other"].pop("Course Identifiers")

# Open ended question 
pre_post_vars["post"]["Course Plan"].remove("post_studyplan_chgw")
pre_post_vars["post"]["Course Plan"].remove("post_studyplan_chgh")

# No extra
pre_post_vars["pre"].pop("Extra")
pre_post_vars["post"].pop("Extra")

# why expected is different from wanted (open) (have the non open version)
pre_post_vars["pre"]["Grade Expectations"].remove("pre_difgrade_o")

# why are you taking this course? (open) (have the non open version)
pre_post_vars["pre"]["Motivation for Taking Course"].remove("pre_reas_o")
# goals for this course (open) (have the non open version)
pre_post_vars["pre"]["Motivation for Taking Course"].remove("pre_coursegoals_o")

# actual completion of other activity i
for i in range(4, 11):
    pre_post_vars["post"]["Other Activities Completion"].remove("post_oactcompx{}".format(i))

    
# What other important activities do you plan on doing in July while completing this course?
for i in range(4, 11):
    pre_post_vars["pre"]["Other Activities List"].remove("pre_oact{}".format(i))
for i in range(11, 16):
    pre_post_vars["post"]["Other Activities List"].remove("post_oact{}".format(i))
    
# importance rank of other activity i
for i in range(4, 11):
    pre_post_vars["pre"]["Other Activities Rank"].remove("pre_oact{}rank".format(i))
    pre_post_vars["post"]["Other Activities Rank"].remove("post_oact{}rank".format(i))
for i in range(11, 16):
    pre_post_vars["post"]["Other Activities Rank"].remove("post_oact{}rank".format(i))
    
# hours per week on other activity i
for i in range(4, 11):
    pre_post_vars["pre"]["Other Activities Time"].remove("pre_oact{}hrs".format(i))
    pre_post_vars["post"]["Other Activities Time"].remove("post_oact{}hrs".format(i))
for i in range(11, 16):
    pre_post_vars["post"]["Other Activities Time"].remove("post_oact{}hrs".format(i))  
    

#how you make performance comparison (open-ended)
pre_post_vars["post"]["Performance Comparisons"].remove("post_perfown_o")

#how others judge your performance (open-ended)
pre_post_vars["post"]["Performance Comparisons"].remove("post_perfoth_o")

#interested in UCI's Online Daily Diary Study
pre_post_vars["pre"]["Study Participation"].remove("pre_studyinterest")

# What type of gift card would you prefer as payment?
pre_post_vars["pre"]["Study Participation"].remove("pre_giftcard")

In [5]:
len([z for k in pre_post_vars for v in pre_post_vars[k] for z in pre_post_vars[k][v]])

234

In [6]:
for k in pre_post_vars:
    print("{}:\tNum construct:  {}\tNum var:  {}".format(
        k, len(pre_post_vars[k]), len([z for v in pre_post_vars[k] for z in pre_post_vars[k][v]])))

post:	Num construct:  30	Num var:  127
pre:	Num construct:  28	Num var:  104
other:	Num construct:  1	Num var:  3


### Weekly

In [7]:
weekly_raw_vars = 'att3w2 att3w3 att3w4 att3w5 cost2w2 cost2w3 cost2w4 cost2w5 cost6w2 cost6w3 cost6w4 cost6w5 cost9w2 cost9w3 cost9w4 cost9w5 wgradecw2 wgradecw3 wgradecw4 wgradecw5 egradecw2 egradecw3 egradecw4 egradecw5 badgradecw2 badgradecw3 badgradecw4 badgradecw5 wgradefw4 wgradefw5 egradefw4 egradefw5 badgradefw4 badgradefw5 wgrademw2 egrademw2 badgrademw2 wgradeqw2 wgradeqw3 wgradeqw4 wgradeqw5 egradeqw2 egradeqw3 egradeqw4 egradeqw5 badgradeqw2 badgradeqw3 badgradeqw4 badgradeqw5 int3w2 int3w3 int3w4 int3w5 iemot1w2 iemot1w3 iemot1w4 iemot1w5 iemot2w2 iemot2w3 iemot2w4 iemot2w5 iemot3w2 iemot3w3 iemot3w4 iemot3w5 iemot4w2 iemot4w3 iemot4w4 iemot4w5 iemot5w2 iemot5w3 iemot5w4 iemot5w5 iemot7w2 iemot7w3 iemot7w4 iemot7w5 woact1 woact2 woact11 woact12 woact21 woact22 woact31 woact32 wcourserank1 woactrank1 woactrank2 wcourserank2 woactrank11 woactrank12 wcourserank3 woactrank21 woactrank22 wcourserank4 woactrank31 woactrank32 wcoursehrs1 woacthrs1 woacthrs2 wcoursehrs2 woacthrs11 woacthrs12 wcoursehrs3 woacthrs21 woacthrs22 wcoursehrs4 woacthrs31 woacthrs32 wcourseplan1 woactplan1 woactplan2 wcourseplan2 woactplan11 woactplan12 wcourseplan3 woactplan21 woactplan22 wcourseplan4 woactplan31 woactplan32 wcoursehrsx3 wcoursereg1 woactreg1 woactreg2 wcoursereg2 woactreg11 woactreg12 wcoursereg3 woactreg21 woactreg22 wcoursereg4 woactreg31 woactreg32 wcoursesat1 woactsat1 woactsat2 wcoursesat2 woactsat11 woactsat12 wcoursesat3 woactsat21 woactsat22 wcoursesat4 woactsat31 woactsat32 util1w2 util1w3 util1w4 util1w5 util4w2 util4w3 util4w4 util4w5 util5w2 util5w3 util5w4 util5w5'.split()

In [8]:
df_week = df["Weekly"]
weekly_vars = defaultdict(lambda: defaultdict(set))
for var in weekly_raw_vars:
    week_suffix = re.findall(r'.*w(\d+)', var)
    if len(week_suffix) > 0:
        weekly_vars[int(week_suffix[0])][df_week[df_week["Var"] == var]["Construct"].tolist()[0]].add(var)
    else:
        con, label = df_week[df_week["Var"] == var][["Construct", "Label"]].values.tolist()[0]
        week_label = re.findall(r'week (\d+)', label)
        if len(week_label) > 0:
            week_from_var = int(re.findall(r'(\d+)', var)[-1]) // 10 + 2 if "oact" in var else \
                            int(re.findall(r'(\d+)', var)[-1]) + 1
#             if week_from_var != int(week_label[0]):
#                 print(var, label)
            weekly_vars[week_from_var][con].add(var)
        else:
            print(var, con, label)  

In [9]:
for w in weekly_vars:
    print("{}:  num construct - {}    num vars - {}\n{}\n".format(
        w, len(weekly_vars[w]), len([k for v in weekly_vars[w] for k in weekly_vars[w][v]]), 
        [(v, len(weekly_vars[w][v])) for v in weekly_vars[w]]))
#     print([k for v in weekly_vars[w] for k in weekly_vars[w][v]])
#     print()

2:  num construct - 16    num vars - 40
[('Attainment Value', 1), ('Cost Value (Emotional)', 1), ('Cost Value (Loss of Valued Alternatives)', 1), ('Cost Value (Outside Effort)', 1), ('Grade Expectations - Course', 3), ('Grade Expectations - Midterm', 3), ('Grade Expectations - Quiz', 3), ('Interest Value', 1), ('Internal/External Motivation', 6), ('Other Activities', 2), ('Other Activities Rank', 3), ('Other Activities Time', 3), ('Plan for Completion', 3), ('Regret for Activity', 3), ('Satisfaction for Activity', 3), ('Utility Value', 3)]

3:  num construct - 15    num vars - 37
[('Attainment Value', 1), ('Cost Value (Emotional)', 1), ('Cost Value (Loss of Valued Alternatives)', 1), ('Cost Value (Outside Effort)', 1), ('Grade Expectations - Course', 3), ('Grade Expectations - Quiz', 3), ('Interest Value', 1), ('Internal/External Motivation', 6), ('Other Activities', 2), ('Other Activities Rank', 3), ('Other Activities Time', 3), ('Plan for Completion', 3), ('Regret for Activity', 3), 

In [11]:
train_var_map = {}
train_var_map["week_1"] = dict(pre_post_vars["pre"])
for i in range(2, 6):
    train_var_map["week_{}".format(i)] = dict(weekly_vars[int(i)])
train_var_map["post"] = dict(pre_post_vars["post"])

In [14]:
# save_object(train_var_map, "../data/var_map.pkl")